In [1]:
from instant_ack import *

%load_ext autoreload
%autoreload 2

2024-09-10 17:53:53.272 | INFO     | instant_ack.config:<module>:11 - PROJ_ROOT path is: /data/instant-ack


# Load data

In [2]:
unique = [
    "server",
    "packet_type",
    "measurement_ts",
]

df_responses = (
    cv.load_data("all-interop-servers")
    .filter(
        # Server responses
        pl.col("udp.srcport") == 443,
        pl.col("quic.ack.ack_delay").is_not_null(),
        # Initial or Handshake packet
        pl.col("quic.long.packet_type").str.contains("0|2"),
    )
    .with_columns(
        # Extract available ack_delay exponenets
        # Before the handshake is finished, the default should be used
        pl.col(["quic.ack.ack_delay", "tls.quic.parameter.ack_delay_exponent"]).cast(
            pl.UInt32
        ),
        packet_type=pl.when(pl.col("quic.long.packet_type").str.contains("0"))
        .then(pl.lit("Initial"))
        .otherwise(pl.lit("Handshake")),
    )
    .with_columns(
        pl.col("tls.quic.parameter.ack_delay_exponent").forward_fill().backward_fill(),
    )
    .with_columns(
        ack_delay_ms=pl.when(
            pl.col("tls.quic.parameter.ack_delay_exponent").is_not_null()
        )
        .then(
            pl.col(
                [
                    "quic.ack.ack_delay",
                ]
            )
            * 2 ** pl.col("tls.quic.parameter.ack_delay_exponent")
        )
        .otherwise(
            # This will always be the case for the first ACK of a connection
            # since he server handshake packet with the parameters is not yet received
            pl.col("quic.ack.ack_delay")
            * 2**3
        )
        / 1000  # raw unit is microseconds,
    )
    .collect()
    .group_by(unique)
    .first()
)
# Grouping by packet_type yield the first handshake packet received

In [3]:
df_requests = (
    cv.load_data("all-interop-servers")
    .filter(
        pl.col("udp.srcport") != 443,
    )
    .with_columns(
        packet_type=pl.when(pl.col("quic.long.packet_type").str.contains("0"))
        .then(pl.lit("Initial"))
        .otherwise(pl.lit("Handshake"))
    )
    .filter(pl.col("ts") == pl.col("ts").min().over(unique))
    .collect()
)
df_responses = df_responses.join(
    df_requests.select("server", "ts", "measurement_ts", "packet_type"),
    how="left",
    on=["server", "measurement_ts", "packet_type"],
    validate="1:1",
    suffix="_sent",
).with_columns(rtt=(pl.col("ts") - pl.col("ts_sent")).dt.total_microseconds() / 1000)

## Limit to required columns

In [46]:
data = df_responses.select(
    "server",
    "measurement_ts",
    "quic.ack.ack_delay",
    "tls.quic.parameter.ack_delay_exponent",
    "ack_delay_ms",
    "rtt",
    "quic.ack.ack_range",
    "quic.ack.first_ack_range",
    "packet_type",
).with_columns(
    pl.col("measurement_ts").replace(
        {
            "2024-08-14T01:43": "1",
            "2024-08-16T01:42": "2",
            "2024-08-15T01:36": "3",
        }
    )
)
data.head()

server,measurement_ts,quic.ack.ack_delay,tls.quic.parameter.ack_delay_exponent,ack_delay_ms,rtt,quic.ack.ack_range,quic.ack.first_ack_range,packet_type
str,str,u32,u32,f64,f64,str,str,str
"""lsquic""","""1""",148,3,1.184,38.775,null,"""0""","""Initial"""
"""nginx""","""2""",0,3,0.0,39.267,null,"""0""","""Initial"""
"""quinn""","""3""",51,3,0.408,37.818,null,"""0""","""Initial"""
"""aioquic""","""3""",412,3,3.296,41.184,null,"""0""","""Initial"""
"""s2n-quic""","""2""",1895,3,15.16,40.057,null,"""0""","""Initial"""


In [44]:
# msquic does not send ACKs in Initial or Handshake packets
msquic = pl.DataFrame(
    {
        "packet_type": ["Initial", "Handshake"],
        "server": ["msquic", "msquic"],
        "ack_delay_ms": [None, None],
    }
).join(data.select("measurement_ts").unique("measurement_ts"), how="cross")

In [47]:
data = pl.concat([data, msquic], how="diagonal")

In [52]:
data.group_by("server", "packet_type", "measurement_ts").agg(
    pl.col("ack_delay_ms").round(1).str.join(", ")
).sort("packet_type", "measurement_ts", descending=[True, False]).pivot(
    on=["packet_type", "measurement_ts"], index="server"
)

server,"{""Initial"",""1""}","{""Initial"",""2""}","{""Initial"",""3""}","{""Handshake"",""1""}","{""Handshake"",""2""}","{""Handshake"",""3""}"
str,str,str,str,str,str,str
"""s2n-quic""","""14.0""","""15.2""","""14.1""",null,null,null
"""mvfst""","""0.8""",null,"""0.7""","""0.2""",null,"""0.1"""
"""ngtcp2""","""0.0""","""0.0""","""0.0""",null,null,null
"""go-x-net""","""0.0""","""0.0""",null,null,null,null
"""quic-go""","""0.0""","""0.0""","""0.0""",null,null,null
…,…,…,…,…,…,…
"""nginx""","""0.0""","""0.0""","""0.0""",null,null,null
"""lsquic""","""1.2""","""1.1""","""1.2""","""0.2""","""0.2""","""0.2"""
"""xquic""","""1.3""","""1.1""","""1.2""",null,"""0.5""","""0.5"""


### Acknowledgment delay of QUIC server implementations: Appendix D

In [54]:
label = "tab:server_ack_delays"
caption = "Acknowledgment Delay of the first acknowledgment received from server in the Initial and Handshake packet number space."


print(
    data.group_by("server", "packet_type", "measurement_ts")
    .agg(pl.col("ack_delay_ms").round(1).str.join(", "))
    .sort("packet_type", "measurement_ts", descending=[True, False])
    .pivot(on=["packet_type", "measurement_ts"], index="server")
    .with_columns(pl.col("*").replace("", pl.lit(None)))
    .sort("server")
    .rename({"server": "Server"})
    .to_pandas()
    .fillna("-")
    .to_latex(label=label, caption=caption, index=False)
)

\begin{table}
\caption{Acknowledgment Delay of the first acknowledgment received from server in the Initial and Handshake packet number space.}
\label{tab:server_ack_delays}
\begin{tabular}{lllllll}
\toprule
Server & {"Initial","1"} & {"Initial","2"} & {"Initial","3"} & {"Handshake","1"} & {"Handshake","2"} & {"Handshake","3"} \\
\midrule
aioquic & 3.3 & 3.4 & 3.3 & - & - & - \\
go-x-net & 0.0 & 0.0 & - & - & - & - \\
haproxy & 1.0 & 1.0 & - & 0.0 & 0.0 & - \\
kwik & 0.0 & 0.0 & 0.0 & - & - & - \\
lsquic & 1.2 & 1.1 & 1.2 & 0.2 & 0.2 & 0.2 \\
msquic & - & - & - & - & - & - \\
mvfst & 0.8 & - & 0.7 & 0.2 & - & 0.1 \\
neqo & 0.0 & 0.0 & - & 0.0 & 0.0 & - \\
nginx & 0.0 & 0.0 & 0.0 & - & - & - \\
ngtcp2 & 0.0 & 0.0 & 0.0 & - & - & - \\
picoquic & 0.8 & 0.7 & 0.8 & - & - & - \\
quic-go & 0.0 & 0.0 & 0.0 & - & - & - \\
quiche & 1.4 & 1.4 & 1.5 & - & - & - \\
quinn & 0.4 & - & 0.4 & - & - & - \\
s2n-quic & 14.0 & 15.2 & 14.1 & - & - & - \\
xquic & 1.3 & 1.1 & 1.2 & - & 0.5 & 0.5 \\
\bottomru

### ACK delay exponent of server implementations

In [ ]:
cv.load_data("all-interop-servers").filter(
    pl.col("tls.quic.parameter.ack_delay_exponent").is_not_null(),
).group_by("server").agg(
    pl.col("tls.quic.parameter.ack_delay_exponent").unique(),
).collect()